In [1]:
import torch
from gensim.models import KeyedVectors

import han
from data_iterator import DataIterator
from utils import load_pickled_obj

In [2]:
dataset = load_pickled_obj("data/amazon_train_sample.pkl")
wv = KeyedVectors.load("embedding/amazon_sample.wv")
vocab_size = len(wv.vocab)
embedding_dim = wv.vector_size
embedding_matrix = wv.vectors

In [73]:
class WordAttention(torch.nn.Module):
    def __init__(self, input_size=100):
        super(WordAttention, self).__init__()
        self.context_vector = torch.nn.Parameter(
            torch.Tensor(input_size, 1)
        )
        self.linear = torch.nn.Linear(in_features=input_size, out_features=input_size)

    def forward(self, input):
        output = torch.tanh(self.linear(input))
        #print("C ", output.shape)
        output = torch.matmul(output, self.context_vector).squeeze()
        #print("D ", output.shape)
        output = torch.nn.functional.softmax(output, dim=-1)
        print("E ", output.shape)
        print("E-bis ", input.shape)
        out = []
        for alpha, h in zip(output, input):
            alpha = alpha.unsqueeze(1).expand_as(h)
            o = alpha * h
            out.append(o)
        return sum(out)

In [74]:
word_encoder = han.WordEncoder(embedding_matrix)
word_attention = WordAttention()
it = iter(DataIterator(dataset, wv.vocab))
labels, batch = next(it)
batch = torch.LongTensor(batch)
batch = batch.permute(1, 0, 2)
i = batch[0]
i = i.permute(1, 0)
#print("A ", i.shape)
output, _ = word_encoder(i, torch.zeros(2, 64, 50))
#print("B ", output.shape)
output = word_attention(output)
print("F ", output.shape)

E  torch.Size([184, 64])
E-bis  torch.Size([184, 64, 100])
F  torch.Size([64, 100])


In [61]:
torch.Tensor([1,2,3]) + torch.Tensor([1,2,3])

tensor([2., 4., 6.])